In [1]:
# IMPORTS 
# Data Analyis, Visualization Imports
import numpy as np
import pandas as pd
import random
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Sklearn
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
# UNIVERSAL PARAMETERS
TEST_SIZE = 0.25
CROSS_VAL_FOLDS = 5
RANDOM_STATE = 42

# FEATURE IMPORTANCE (will drop features, and re-run analysis on dataset)
USE_FEATURE_IMPORTANCE = False

# Uses previous case's label as a feature (which means it cant generate competition output)
# So leave this False unless you really want to use it.
USE_PREV_CASES = False

# COMPETITION RESULT OUTPUT 
COMPETITION_RESULTS_OUTPUT_NAME = "competition-results"
EXPORT_RESULT_TO_GSHEET = False

In [3]:
# FEATURE KEYS
ORIG_FEATURE_KEYS = ['city', 'year', 'weekofyear', 'week_start_date', 'ndvi_ne',
                     'ndvi_nw','ndvi_se', 'ndvi_sw', 'precipitation_amt_mm',
                     'reanalysis_air_temp_k','reanalysis_avg_temp_k',
                     'reanalysis_dew_point_temp_k','reanalysis_max_air_temp_k',
                     'reanalysis_min_air_temp_k',
                     'reanalysis_precip_amt_kg_per_m2',
                     'reanalysis_relative_humidity_percent',
                     'reanalysis_sat_precip_amt_mm',
                     'reanalysis_specific_humidity_g_per_kg',
                     'reanalysis_tdtr_k','station_avg_temp_c',
                     'station_diur_temp_rng_c',
                     'station_max_temp_c','station_min_temp_c',
                     'station_precip_mm']
NUMERIC_FEATURE_KEYS = ['year', 'weekofyear', 'ndvi_ne', 'ndvi_nw',
                        'ndvi_se', 'ndvi_sw', 'precipitation_amt_mm',
                        'reanalysis_air_temp_k', 'reanalysis_avg_temp_k',
                        'reanalysis_dew_point_temp_k',
                        'reanalysis_max_air_temp_k',
                        'reanalysis_min_air_temp_k',
                        'reanalysis_precip_amt_kg_per_m2',
                        'reanalysis_relative_humidity_percent',
                        'reanalysis_sat_precip_amt_mm',
                        'reanalysis_specific_humidity_g_per_kg',
                        'reanalysis_tdtr_k', 'station_avg_temp_c',
                        'station_diur_temp_rng_c', 'station_max_temp_c',
                        'station_min_temp_c', 'station_precip_mm']
TIME_FEATURE_KEYS = ['week_start_date']
DROP_FEATURE_KEYS = ['precipitation_amt_mm', 'reanalysis_air_temp_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm']
CATEGORY_KEYS = ['city']

# added new feature, also week of year is now a category
PIPELINE_CATEGORY_KEYS = ['city', 'weekofyear', 'yearofdecade']  # if treating week and year as categories

PIPELINE_CATEGORY_KEYS_RF = ['city'] 
PIPELINE_NUMERIC_KEYS = ['reanalysis_avg_temp_k',
                        'reanalysis_dew_point_temp_k',
                        'reanalysis_max_air_temp_k',
                        'reanalysis_min_air_temp_k',
                        'reanalysis_precip_amt_kg_per_m2',
                        'reanalysis_relative_humidity_percent',
                        'reanalysis_sat_precip_amt_mm',
                        'reanalysis_specific_humidity_g_per_kg',
                        'reanalysis_tdtr_k']
PIPELINE_NUMERIC_KEYS_RF = []

# ndvi_xxx are already scaled to [-1.0,1.0], so excluding from pipeline
PIPELINE_PASSTHRU_KEYS = ['ndvi_ne', 'ndvi_nw','ndvi_se', 'ndvi_sw']

# if using RF, none of the numerical values need scaling, so passthru.
PIPELINE_PASSTHRU_KEYS_RF = ['reanalysis_avg_temp_k',
                        'reanalysis_dew_point_temp_k',
                        'reanalysis_max_air_temp_k',
                        'reanalysis_min_air_temp_k',
                        'reanalysis_precip_amt_kg_per_m2',
                        'reanalysis_relative_humidity_percent',
                        'reanalysis_sat_precip_amt_mm',
                        'reanalysis_specific_humidity_g_per_kg',
                        'reanalysis_tdtr_k', 'cos_weekofyear', 'cos_yearofdecade','ndvi_ne', 'ndvi_nw','ndvi_se', 'ndvi_sw']
PIPELINE_DROP_KEYS = ['year','week_start_date',  'weekofyear', 'yearofdecade']  #FOR REFERENCE ONLY, PIPELINE IMPLICITLY DROPS THESE

# LABEL KEYS
ORIG_LABEL_KEYS = ['city', 'year', 'weekofyear', 'total_cases'] 
NUMERIC_LABEL_KEYS = ['year','weekofyear','total_cases']
DROP_LABEL_KEYS = ['city', 'year','weekofyear']

In [4]:
def clean_data(input_df, drop_keys=[], numeric_keys=[], cat_keys=[], time_keys=[]):
    new_df = input_df.copy()
  
    # Drop keys
    drop_key_list = intersection(new_df.keys(), drop_keys)
    new_df = new_df.drop(columns=drop_key_list)

    curr_key_list = new_df.keys()

    # Convert to numeric and time
    numeric_key_list = intersection(curr_key_list, numeric_keys)
    new_df[numeric_key_list] = new_df[numeric_key_list].apply(pd.to_numeric)

    time_key_list = intersection(curr_key_list, time_keys)
    new_df[time_key_list] = new_df[time_key_list].apply(pd.to_datetime)
  
    # Insert category processing --- placeholder, no processing performed
    cat_key_list = intersection(curr_key_list, cat_keys)
  
    return new_df

def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

def clean_training_data(input_df, drop_keys=[], numeric_keys=[], cat_keys=[], time_keys=[]):
    # First generically clean by dropping unneeded keys, converting numeric, time values etc
    raw_X = clean_data(input_df, drop_keys=drop_keys, numeric_keys=numeric_keys, cat_keys=cat_keys, time_keys=time_keys)

    # Separate out sj and iq first, then perform imputing, so that it doesnt accidentally use the other city's data for linear interpolation
    X_sj = clean_data(raw_X.loc[raw_X['city'] == 'sj'])
    X_sj = impute_empty_cells(X_sj) 

    X_iq = clean_data(raw_X.loc[raw_X['city'] == 'iq'])
    X_iq = impute_empty_cells(X_iq) 

    # final X to be used for further analysis
    basic_X = X_sj.append(X_iq)
    return basic_X


In [5]:
def impute_empty_cells(input_df):
    out_df = input_df.copy()
    for column in input_df:
        # indices in the column which hard zeroes or NaN
        null_or_zero_indices = return_empty_cells_in_1d_array(input_df[column], missing_vals=[0])
    
        for index in null_or_zero_indices:
            min_index = input_df[column].index.values[0]
            max_index = input_df[column].index.values[len(input_df[column])-1]
            imputing_indices = return_indices_for_interpolation(index, min_index, max_index, null_or_zero_indices)
      
            imputed_value  = simple_linear_interpolation(input_df[column], index, imputing_indices)
      
            out_df.at[index,column] = imputed_value
      
    return out_df


def simple_linear_interpolation(input_df, index, imputing_indices):
    model = LinearRegression()
  
    X = np.asarray(imputing_indices).reshape(len(imputing_indices),1)
    y = input_df[imputing_indices].to_numpy().reshape(len(imputing_indices),1)

    model.fit(X, y)
  
    x_new = np.asarray(int(index)).reshape(1,1)
    y_new = model.predict(x_new)

    return y_new[0][0]

def return_empty_cells_in_1d_array(input_df, missing_vals=[]):
    column_array = input_df
    missing_val_bool = column_array.isna()
    for val in missing_vals:
        missing_val_bool = missing_val_bool | (column_array == val)
  
    null_or_zero_indices = column_array[missing_val_bool].index.tolist() 
    return null_or_zero_indices

def return_indices_for_interpolation(index, min_index, max_index, null_or_zero_indices, values_to_interpolate=2):
    index_list = []
    values_needed_left = 1
    values_needed_right = 1
    curr_index = index
  
    while (values_needed_left<= values_to_interpolate) and (curr_index >= min_index):
        if curr_index not in null_or_zero_indices:
            index_list.append(curr_index)
            values_needed_left +=1
        curr_index-=1
    curr_index = index
    while (values_needed_right<= values_to_interpolate) and (curr_index <= max_index):
        if curr_index not in null_or_zero_indices:
            index_list.append(curr_index)
            values_needed_right +=1
        curr_index+=1
    return index_list

In [6]:
def output_yearofdecade(input_df):
    true_year = (input_df - pd.to_timedelta(input_df.dt.weekday, unit='D')).dt.year
    yearofdecade = true_year.mod(10).astype(int)
    return yearofdecade

In [7]:
def get_column_names_from_ColumnTransformer(column_transformer):
    col_name = []
    for transformer_in_columns in column_transformer.transformers_[:-1]:#the last transformer is ColumnTransformer's 'remainder' which we ignore, since those are dropped.
        raw_col_name = transformer_in_columns[2]
        if isinstance(transformer_in_columns[1],Pipeline): 
            transformer = transformer_in_columns[1].steps[-1][1]
        else:
            transformer = transformer_in_columns[1]
        try:
            names = transformer.get_feature_names()
        except AttributeError: # if no 'get_feature_names' function, use raw column name
            names = raw_col_name
        if isinstance(names,np.ndarray): 
            col_name += names.tolist()
        elif isinstance(names,list):
            col_name += names
        elif isinstance(names,str):
            col_name.append(names)
    return col_name

In [8]:
def show_feature_importance(X,y,params_dict=None,most_important_n=-1, print_all=False):
    model = RandomForestRegressor()
    if params_dict is not None:
        model.set_params(**params_dict)
    model.fit(X,y)
    importances = list(zip(list(X), model.feature_importances_))
    sorted_importances = sorted(importances, key = lambda x: x[1]) 
    sorted_importances.reverse()
    sorted_importances_df = pd.DataFrame(sorted_importances, columns=["Feature","Importance"])
    sorted_importances_df.index+=1
    sorted_importances_df["Cumulative Importance"] = sorted_importances_df["Importance"].cumsum()
    if print_all or most_important_n > sorted_importances_df.shape[0]:
        with pd.option_context("display.max_rows", 1000):
            display(sorted_importances_df)
    else:
        with pd.option_context("display.max_rows", 1000):
            display(sorted_importances_df.head(most_important_n))

    feature_list, _ = zip(*sorted_importances)
    if most_important_n == -1 or most_important_n > len(feature_list):
        return list(feature_list)
    else:
        return list(feature_list[:most_important_n])

In [9]:
def predictions_and_mae(estimator, X, y_true):
    y_pred = estimator.predict(X)
    return y_pred, mean_absolute_error(y_true, y_pred)

In [10]:
features_train_df = pd.read_csv("../dengue_fever/data/dengue_features_train.csv")
labels_train_df = pd.read_csv("../dengue_fever/data/dengue_labels_train.csv")

In [11]:
labels_competition_df = pd.read_csv('../dengue_fever/data/submission_format.csv')
features_competition_df = pd.read_csv('../dengue_fever/data/dengue_features_test.csv')

In [12]:
basic_X = clean_training_data(features_train_df, drop_keys=DROP_FEATURE_KEYS, numeric_keys=NUMERIC_FEATURE_KEYS, time_keys=TIME_FEATURE_KEYS)
basic_y = clean_data(labels_train_df, numeric_keys=NUMERIC_LABEL_KEYS)

competition_basic_X = clean_training_data(features_competition_df, drop_keys=DROP_FEATURE_KEYS, numeric_keys=NUMERIC_FEATURE_KEYS, time_keys=TIME_FEATURE_KEYS)
competition_basic_y = clean_data(labels_competition_df, numeric_keys=NUMERIC_LABEL_KEYS)

/var/folders/fj/g8m8pm2j58g6_6gdtyfpn1640000gn/T/ipykernel_33104/731150205.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  basic_X = X_sj.append(X_iq)
/var/folders/fj/g8m8pm2j58g6_6gdtyfpn1640000gn/T/ipykernel_33104/731150205.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  basic_X = X_sj.append(X_iq)


In [13]:
visual_df_sj = basic_y[basic_y['city'] == 'sj']
visual_df_iq = basic_y[basic_y['city'] == 'iq']

In [14]:
X = basic_X.copy()
y = basic_y  #.copy()  (no further steps occuring to y's)

competition_X = competition_basic_X.copy()
competition_y = competition_basic_y #.copy()  (no further steps occuring to y)

In [15]:
X['yearofdecade'] = output_yearofdecade(basic_X['week_start_date'])
competition_X['yearofdecade'] = output_yearofdecade(basic_X['week_start_date'])

In [16]:
# Test, trying to handle the periodicity myself by using Cosines of the year and week

max_year_of_decade = 9.0
max_weekofyear = 53.0
X['cos_yearofdecade'] = np.cos(2*np.pi*X['yearofdecade']/max_year_of_decade)
competition_X['cos_yearofdecade'] = np.cos(2*np.pi*competition_X['yearofdecade']/max_year_of_decade)

X['cos_weekofyear'] = np.cos(2*np.pi*X['weekofyear']/max_weekofyear)
competition_X['cos_weekofyear'] = np.cos(2*np.pi*competition_X['weekofyear']/max_weekofyear)

In [17]:
if USE_PREV_CASES:
    X['previous_cases'] = y['total_cases']
    X['previous_cases']= X['previous_cases'].shift(periods=1)
    X.at[1,'previous_cases']=X['previous_cases'][2]

In [18]:
cat_attribs = PIPELINE_CATEGORY_KEYS_RF
# num_attribs = PIPELINE_NUMERIC_KEYS  # if num_attribs need scaling
num_attribs = PIPELINE_NUMERIC_KEYS_RF # this is actually empty, because RF Numerics dont need scaling.
passthru_attribs = PIPELINE_PASSTHRU_KEYS_RF.copy()

if USE_PREV_CASES:
    passthru_attribs.append('previous_cases')
drop_attribs = PIPELINE_DROP_KEYS  # for reference only, nothing is being done to ['year','week_start_date'] so it will be dropped

full_pipeline = ColumnTransformer([
                                   ("cat",OneHotEncoder(sparse=False, drop='first'), cat_attribs),
                                   ("num", StandardScaler(), num_attribs),
                                   ("passthru",'passthrough', passthru_attribs),
                                   ],
                                  remainder='drop'
                                  )
full_pipeline.fit(X)
pipeline_cols = get_column_names_from_ColumnTransformer(full_pipeline)

/Users/Nikki/.pyenv/versions/3.10.6/envs/dengue_fever/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [19]:
# ensures X_prep remains a dataframe 
X_prep = pd.DataFrame(full_pipeline.transform(X), columns=pipeline_cols)
y_prep = y # no need to deep copy, no changes were made
X_prep.index = np.arange(1, len(X_prep) + 1)


if not USE_PREV_CASES:
    competition_X_prep = pd.DataFrame(full_pipeline.transform(competition_X), columns=pipeline_cols)
    competition_X_prep.index = np.arange(1, len(competition_X_prep) + 1)
    competition_y_prep = competition_y #no need to deep copy, no changes were made

In [20]:
X_prep.shape, y_prep.shape


((1456, 16), (1456, 4))

In [21]:
X_train, X_test,y_train, y_test = train_test_split(X_prep,y_prep['total_cases'],test_size=TEST_SIZE,random_state=RANDOM_STATE)

In [22]:
prelim_important_feature_list = show_feature_importance(X_train,y_train, most_important_n=15, print_all=False, params_dict={'n_estimators':100})
prelim_important_feature_list


,Feature,Importance,Cumulative Importance
1,ndvi_sw,0.219270,0.219270
2,reanalysis_min_air_temp_k,0.133517,0.352787
3,ndvi_nw,0.119429,0.472215
4,cos_weekofyear,0.081604,0.553820
5,cos_yearofdecade,0.076800,0.630619
6,reanalysis_tdtr_k,0.048323,0.678942
7,reanalysis_dew_point_temp_k,0.044449,0.723391
8,reanalysis_sat_precip_amt_mm,0.044383,0.767774
9,reanalysis_precip_amt_kg_per_m2,0.040257,0.808031
10,reanalysis_specific_humidity_g_per_kg,0.039075,0.847106


['ndvi_sw',
 'reanalysis_min_air_temp_k',
 'ndvi_nw',
 'cos_weekofyear',
 'cos_yearofdecade',
 'reanalysis_tdtr_k',
 'reanalysis_dew_point_temp_k',
 'reanalysis_sat_precip_amt_mm',
 'reanalysis_precip_amt_kg_per_m2',
 'reanalysis_specific_humidity_g_per_kg',
 'reanalysis_relative_humidity_percent',
 'reanalysis_avg_temp_k',
 'ndvi_ne',
 'ndvi_se',
 'reanalysis_max_air_temp_k']

In [23]:
tree_reg = DecisionTreeRegressor(max_depth=4)

forest_reg = RandomForestRegressor(random_state=RANDOM_STATE)
forest_reg.fit(X_train, y_train)

lin_reg = LinearRegression()

lin_svm_reg = LinearSVR()
svr_reg = SVR()

In [24]:
# Cross validate
tree_reg_scores = cross_val_score(tree_reg, X_train, y_train, cv=CROSS_VAL_FOLDS, scoring='neg_mean_absolute_error')
print("Decision Tree score:", tree_reg_scores)

forest_reg_scores = cross_val_score(forest_reg, X_train, y_train, cv=CROSS_VAL_FOLDS, scoring='neg_mean_absolute_error')
print("Random Forest score: ", forest_reg_scores)

lin_reg_scores = cross_val_score(lin_reg, X_train, y_train, cv=CROSS_VAL_FOLDS, scoring='neg_mean_absolute_error')
print("Linear Reg score: ", lin_reg_scores)

lin_svm_reg_scores = cross_val_score(lin_svm_reg, X_train, y_train, cv=CROSS_VAL_FOLDS, scoring='neg_mean_absolute_error')
print("Linear SVM score: ", lin_svm_reg_scores)

svr_reg_scores = cross_val_score(svr_reg, X_train, y_train, cv=CROSS_VAL_FOLDS, scoring='neg_mean_absolute_error')
print("SVR score: ", svr_reg_scores)

Decision Tree score: [-15.07575587 -20.99447252 -20.01862523 -15.43054614 -14.98199479]
Random Forest score:  [-16.56557078 -18.22415525 -16.71844037 -13.63376147 -13.61619266]
Linear Reg score:  [-20.71414154 -20.40224407 -20.13015454 -19.0308359  -17.41662154]


/Users/Nikki/.pyenv/versions/3.10.6/envs/dengue_fever/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/Nikki/.pyenv/versions/3.10.6/envs/dengue_fever/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/Nikki/.pyenv/versions/3.10.6/envs/dengue_fever/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/Nikki/.pyenv/versions/3.10.6/envs/dengue_fever/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/Nikki/.pyenv/versions/3.10.6/envs/dengue_fever/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to co

Linear SVM score:  [-24.15875914 -21.43330537 -19.42670744 -16.33706024 -29.96169854]
SVR score:  [-18.69372582 -20.91744305 -21.30452375 -16.13707533 -16.37343949]


In [25]:
y_pred = forest_reg.predict(X_train)
forest_reg_mae = mean_absolute_error(y_train, y_pred)

y_pred, y_train
print(forest_reg_mae)

5.587591575091575


In [26]:
svr_reg = SVR()
svr_reg_scores = cross_val_score(svr_reg, X_train, y_train, cv=CROSS_VAL_FOLDS, scoring='neg_mean_absolute_error')
print("SVR score: ", svr_reg_scores)

SVR score:  [-18.69372582 -20.91744305 -21.30452375 -16.13707533 -16.37343949]


In [27]:
svr_reg.fit(X_train, y_train)
print("The SVM's MAE on training data is:",  predictions_and_mae(svr_reg, X_train, y_train)[1])

The SVM's MAE on training data is: 18.6022820081744


In [28]:
sv_param_grid = {'kernel': ['linear','rbf','sigmoid'],
                 'tol': [0.0001, 0.001, 0.01, 0.1],
                 'gamma': ['scale','auto'],
                 'degree': [2,3]
                 }

sv_grid_search = GridSearchCV(SVR(), sv_param_grid, cv=CROSS_VAL_FOLDS, scoring='neg_mean_absolute_error', return_train_score=True)
sv_grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'degree': [2, 3], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'sigmoid'],
                         'tol': [0.0001, 0.001, 0.01, 0.1]},
             return_train_score=True, scoring='neg_mean_absolute_error')

In [29]:
sv_grid_search.best_params_

{'degree': 2, 'gamma': 'scale', 'kernel': 'linear', 'tol': 0.1}

In [30]:
sv_grid_search.best_estimator_

SVR(degree=2, kernel='linear', tol=0.1)

In [31]:
sv_grid_train_pred, sv_mae_grid_train = predictions_and_mae(sv_grid_search.best_estimator_, X_train, y_train)
print("The grid-search optimized random forest model's MAE on training data is:", sv_mae_grid_train )

The grid-search optimized random forest model's MAE on training data is: 16.237704682940343


In [32]:
# Creating and fitting the model with 10 trees
rf_reg = RandomForestRegressor(n_estimators=10, random_state=RANDOM_STATE, criterion='mae')
rf_reg.fit(X_train, y_train)

/Users/Nikki/.pyenv/versions/3.10.6/envs/dengue_fever/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:407: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  warn(


RandomForestRegressor(criterion='mae', n_estimators=10, random_state=42)

In [33]:
rf_reg_scores = cross_val_score(rf_reg, X_train, y_train, cv=CROSS_VAL_FOLDS, scoring='neg_mean_absolute_error')
print("Random Forest score: ", rf_reg_scores)

/Users/Nikki/.pyenv/versions/3.10.6/envs/dengue_fever/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:407: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  warn(
/Users/Nikki/.pyenv/versions/3.10.6/envs/dengue_fever/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:407: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  warn(
/Users/Nikki/.pyenv/versions/3.10.6/envs/dengue_fever/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:407: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  warn(
/Users/Nikki/.pyenv/versions/3.10.6/envs/dengue_fever/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:407: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in ver

Random Forest score:  [-20.3760274  -17.69634703 -19.98004587 -16.11284404 -17.09724771]


In [34]:
print("The random forest model's MAE on training data is:",  predictions_and_mae(rf_reg, X_train, y_train)[1])

The random forest model's MAE on training data is: 6.9259615384615385


In [35]:
max_features = int(len(X_train.keys()))
half_max_features = int(round(0.5*max_features))
quarter_max_features = int(round(0.25*max_features))

In [36]:
rf_param_grid = {'n_estimators': [50, 100, 200,300,400,500],
                 'bootstrap': [True, False],
                 'max_features': [quarter_max_features, half_max_features, max_features],
                 #'max_features': [2,4,6,8,10,12,14,16],
                 'min_samples_leaf': [2,3,4,5,10],
                 }

rf_grid_search = GridSearchCV(RandomForestRegressor(random_state=RANDOM_STATE), rf_param_grid, cv=CROSS_VAL_FOLDS, scoring='neg_mean_absolute_error', return_train_score=True)
rf_grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
             param_grid={'bootstrap': [True, False], 'max_features': [4, 8, 16],
                         'min_samples_leaf': [2, 3, 4, 5, 10],
                         'n_estimators': [50, 100, 200, 300, 400, 500]},
             return_train_score=True, scoring='neg_mean_absolute_error')

In [37]:
rf_grid_search.best_params_

{'bootstrap': False,
 'max_features': 8,
 'min_samples_leaf': 3,
 'n_estimators': 500}

In [38]:
rf_grid_search.best_estimator_

RandomForestRegressor(bootstrap=False, max_features=8, min_samples_leaf=3,
                      n_estimators=500, random_state=42)

In [39]:
rf_grid_train_pred, mae_grid_train = predictions_and_mae(rf_grid_search.best_estimator_, X_train, y_train)
print("The grid-search optimized random forest model's MAE on training data is:", mae_grid_train )

The grid-search optimized random forest model's MAE on training data is: 3.848423260073256


In [40]:
rnd_max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
rnd_max_depth.append(None)

rf_param_rnd = {'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)],
                'bootstrap': [True, False],
                'max_features': ['auto', 'sqrt'],
                'min_samples_leaf': [1, 2, 3, 4, 5,6,7,8,9,10,20,30],
                'max_depth': rnd_max_depth,
                'min_samples_split': [2, 3, 4, 5, 6, 8, 10,15],
                
                 }

rf_rnd_search = RandomizedSearchCV(RandomForestRegressor(random_state=RANDOM_STATE), rf_param_rnd, cv=CROSS_VAL_FOLDS, scoring='neg_mean_absolute_error', return_train_score=True)
rf_rnd_search.fit(X_train, y_train)

print(rf_rnd_search.best_params_)
_, rnd_mae_grid_train = predictions_and_mae(rf_rnd_search.best_estimator_, X_train, y_train)
print("The grid-search optimized random forest model's MAE on training data is:", rnd_mae_grid_train )


_, rnd_test_mae = predictions_and_mae(rf_rnd_search.best_estimator_, X_test, y_test)
print("The best model's MAE on test data is:",rnd_test_mae)

/Users/Nikki/.pyenv/versions/3.10.6/envs/dengue_fever/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/Nikki/.pyenv/versions/3.10.6/envs/dengue_fever/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/Nikki/.pyenv/versions/3.10.6/envs/dengue_fever/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past

{'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 80, 'bootstrap': False}
The grid-search optimized random forest model's MAE on training data is: 2.0582346357346357
The best model's MAE on test data is: 16.959062881562883


In [41]:
# Change this if rf_grid was not the best choice here
best_model_name = RandomForestRegressor
best_estimator = rf_grid_search.best_estimator_
best_params = rf_grid_search.best_params_
best_param_grid = rf_param_grid  # Only to be used if re-doing feature importance (below)

In [42]:
y_test_pred, test_mae = predictions_and_mae(best_estimator, X_test, y_test)

print("The best model's MAE on test data is:",test_mae)

The best model's MAE on test data is: 16.00112335164835


In [77]:
# This shows that approx n=12 features suffices for >90% of the importance.
most_important_feature_list=show_feature_importance(X_train,y_train, most_important_n=12, print_all=True, params_dict=best_params)

,Feature,Importance,Cumulative Importance
1,ndvi_sw,0.199572,0.199572
2,ndvi_nw,0.135440,0.335012
3,cos_yearofdecade,0.108559,0.443571
4,cos_weekofyear,0.098121,0.541692
5,reanalysis_min_air_temp_k,0.087326,0.629018
6,reanalysis_dew_point_temp_k,0.059436,0.688454
7,reanalysis_specific_humidity_g_per_kg,0.059169,0.747624
8,reanalysis_tdtr_k,0.045835,0.793459
9,ndvi_se,0.040080,0.833539
10,ndvi_ne,0.030062,0.863601


In [78]:
# Features 
feature_opt_passthru_attribs = most_important_feature_list

# Pipeline
feature_opt_pipeline = ColumnTransformer([
                                          ("passthru",'passthrough', feature_opt_passthru_attribs),
                                          ],
                                         remainder='drop'
                                         )
feature_opt_pipeline.fit(X_prep)
feature_opt_pipeline_cols = get_column_names_from_ColumnTransformer(feature_opt_pipeline)


feature_opt_X_prep = pd.DataFrame(feature_opt_pipeline.transform(X_prep), columns=feature_opt_pipeline_cols)
feature_opt_X_prep.index = np.arange(1, len(feature_opt_X_prep) + 1)
feature_opt_y_prep = y # no need to deep copy, no changes were made

if not (USE_PREV_CASES):
    feature_opt_competition_X_prep = pd.DataFrame(feature_opt_pipeline.transform(competition_X_prep), columns=feature_opt_pipeline_cols)
    feature_opt_competition_y_prep = competition_y #no need to deep copy, no changes were made

# Train Test Split
feature_opt_X_train, feature_opt_X_test, feature_opt_y_train, feature_opt_y_test = train_test_split(feature_opt_X_prep,feature_opt_y_prep['total_cases'],test_size=TEST_SIZE,random_state=RANDOM_STATE)

# Model Fitting
feature_opt_model_name = best_model_name
max_features = int(len(feature_opt_X_train.keys()))
half_max_features = int(round(0.5*max_features))
quarter_max_features = int(round(0.25*max_features))


feature_opt_param_grid = {'n_estimators': [50, 100, 200,300],
                 'bootstrap': [False],
                 'max_features': [quarter_max_features, half_max_features, max_features],
                 'min_samples_leaf': [2,3,4],
                 }

feature_opt_grid_search =  GridSearchCV(feature_opt_model_name(random_state=RANDOM_STATE), feature_opt_param_grid, cv=CROSS_VAL_FOLDS, scoring='neg_mean_squared_error', return_train_score=True)
feature_opt_grid_search.fit(feature_opt_X_train, feature_opt_y_train)

feature_opt_estimator = feature_opt_grid_search.best_estimator_
feature_opt_params = feature_opt_grid_search.best_params_

feature_opt_grid_train_pred, feature_opt_mae_grid_train = predictions_and_mae(feature_opt_grid_search.best_estimator_, feature_opt_X_train, feature_opt_y_train)
print("The feature optimized best model grid-search's MAE on training data is:", feature_opt_mae_grid_train )

feature_opt_y_test_pred, feature_opt_test_mae = predictions_and_mae(feature_opt_grid_search.best_estimator_, feature_opt_X_test, feature_opt_y_test)
print("The best model's MAE on test data is:",feature_opt_test_mae)

The feature optimized best model grid-search's MAE on training data is: 4.000129629629629
The best model's MAE on test data is: 15.755440934065938


In [79]:
if USE_FEATURE_IMPORTANCE:
    final_model_name = feature_opt_model_name
    final_params = feature_opt_params
    final_X_prep = feature_opt_X_prep
    final_y_prep = feature_opt_y_prep
    if not USE_PREV_CASES:
        final_competition_X_prep = feature_opt_competition_X_prep
        final_competition_y_prep = feature_opt_competition_y_prep
else:
    final_model_name = best_model_name
    final_params = best_params
    final_X_prep = X_prep
    final_y_prep = y_prep
    if not USE_PREV_CASES:
        final_competition_X_prep = competition_X_prep
        final_competition_y_prep = competition_y_prep

In [80]:
final_model = final_model_name(**final_params)
final_estimator = final_model.fit(final_X_prep,final_y_prep['total_cases'])

In [81]:
y_full_train_pred, full_train_mae = predictions_and_mae(final_estimator, final_X_prep,final_y_prep['total_cases'])
print("The final model's MAE on full training data is:",  full_train_mae)

The final model's MAE on full training data is: 4.024301785714282


In [82]:
if not USE_PREV_CASES:
    y_competition_pred,_ = predictions_and_mae(final_estimator, final_competition_X_prep, final_competition_y_prep['total_cases'])

In [1]:
submission_df

NameError: name 'submission_df' is not defined

In [2]:
submission_df = competition_y.copy()
if not USE_PREV_CASES:
    submission_df['total_cases'] = y_competition_pred
    submission_df['total_cases'] = submission_df['total_cases'].astype(int)

NameError: name 'competition_y' is not defined

In [116]:
sj_pred = submission_df[submission_df['city'] == 'sj']
iq_pred = submission_df[submission_df['city'] == 'iq']

In [117]:
sj_pred = sj_pred["total_cases"].to_numpy()
iq_pred = iq_pred["total_cases"].to_numpy()

In [127]:
sj = []
for i in sj_pred:
    sj.append([i])

In [125]:
iq = []
for i in iq_pred:
    iq.append([i])

In [112]:
submission = pd.read_csv('../dengue_fever/data/submission_format.csv',
                            index_col=[0, 1, 2])

In [128]:
submission.total_cases = np.concatenate([sj, iq])
submission.to_csv("../dengue_fever/data/ak_submission_ARIMA_new.csv")

## BONUS

In [ ]:
lookback_train_X = X_prep.copy()
lookback_train_y = y_prep

lookback_train_X.index  = np.arange(1, len(lookback_train_X) + 1)
lookback_train_X['previous_cases'] = lookback_train_y['total_cases']
lookback_train_X['previous_cases']= lookback_train_X['previous_cases'].shift(periods=1)
lookback_train_X.at[1,'previous_cases']=lookback_train_X['previous_cases'][2]


# Model Fitting
lookback_model_name = best_model_name
max_features = int(len(lookback_train_X.keys()))
half_max_features = int(round(0.5*max_features))
quarter_max_features = int(round(0.25*max_features))

lookback_param_grid = {'n_estimators': [50, 100, 200,300],
                 'bootstrap': [False],
                 'max_features': [quarter_max_features, half_max_features, max_features],
                 'min_samples_leaf': [2,3,4],
                 }

lookback_grid_search =  GridSearchCV(lookback_model_name(random_state=RANDOM_STATE), lookback_param_grid, cv=CROSS_VAL_FOLDS, scoring='neg_mean_squared_error', return_train_score=True)
lookback_grid_search.fit(lookback_train_X, lookback_train_y['total_cases'])

lookback_estimator = lookback_grid_search.best_estimator_
lookback_params = lookback_grid_search.best_params_

In [ ]:
lookback_test_X = final_competition_X_prep.copy()
lookback_test_X.index  = np.arange(1, len(lookback_test_X) + 1)

lookback_test_X['previous_cases'] = y_competition_pred
lookback_test_X['previous_cases']= lookback_test_X['previous_cases'].shift(periods=1)
lookback_test_X.at[1,'previous_cases']=lookback_test_X['previous_cases'][2]

In [52]:
lookback_competition_pred, lookback_vs_nolookback_mae = predictions_and_mae(lookback_estimator, lookback_test_X, y_competition_pred)
